# Clinical Profiles Calculations
### Developed by Steph Howson, JHU/APL Data Scientist


Clinical Profiles are meant to be a reference dataset for researchers and data scientists. The expansion of this framework to other datasets is a meaningful way to define clinically relevant distributions. This breadth and depth of statistical calculations has been previously unavailable to the open source community. By open sourcing this code and the reference dataset, the NCATS Clinical Profiles team is hoping to inspire discovery and discussion within the clinical data science community.

## Import Necessary Libraries & Connect to Database in PMAP Microsoft Azure

In [1]:
import os
import sqlalchemy
import urllib.parse
import pandas as pd
import numpy as np
import getpass
from SciServer import Authentication
from fhir.resources.clinicalprofile import *
from fhir.resources.coding import Coding
from datetime import datetime
import json
import fhir_loader

In [2]:
user = 'clinicalprofile'
passwd = getpass.getpass('Password for '+user+':')

Password for clinicalprofile:········


In [3]:
driver='FreeTDS'
tds_ver='8.0'

host_ip='esmpmdbdev6.database.windows.net'
db_port=1433
db='ClinicalProfile'

conn_str=('DRIVER={};SERVER={};PORT={};DATABASE={};UID={};PWD={};TDS_VERSION={}'.format(
driver, host_ip, db_port, db, user, passwd, tds_ver))


In [4]:
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect='+urllib.parse.quote(conn_str))

## Get tables from PMAP to do analysis

In [5]:
# Tables to use: vw_pc_dm_demo, DIAGNOSIS, ENCOUNTER, ICD10, ICD9, LAB_RESULT_CM, PRESCRIBING, PROCEDURES

In [5]:
query="""select * from [dbo].[vw_pc_dm_demo]"""
df_demographics = pd.read_sql_query(query, engine)

In [6]:
df_demographics['race_code'] = df_demographics.RACE.map({'01':'Other','02':'Other','03':'Black or African American',
                                                            '04':'Other','05':'White or Caucasian','06':'Other',
                                                            '07':'Other','NI':'Other','UN':'Other','OT':'Other'})

In [8]:
# Users can upload files that exist on Desktop and laod into environment, if needed
# Cohorts defined by CDC
cohortDefinitions = pd.read_csv('DiabetesCohorts.csv')

In [9]:
query = ("""select demo.PATID, ENCOUNTERID, LAB_LOINC, RESULT_DATE, RESULT_NUM, RAW_UNIT
            from [dbo].[LAB_RESULT_CM] as lab
         inner join [dbo].[vw_pc_dm_demo] as demo
         on lab.PATID = demo.PATID """)
df_labs = pd.read_sql_query(query, engine)

In [10]:
query = ("""select * from [dbo].[jh_loinc] """)
df_loincinfo = pd.read_sql_query(query, engine)

In [11]:
df_labs = df_labs.merge(df_loincinfo, how='left', left_on='LAB_LOINC', right_on='Loinc_Code')

In [12]:
query = ("""select demo.PATID, RX_START_DATE, JH_INGREDIENT_RXNORM_CODE
            from [dbo].[PRESCRIBING] as med
         inner join [dbo].[vw_pc_dm_demo] as demo
         on med.PATID = demo.PATID """)
df_meds = pd.read_sql_query(query, engine)

In [13]:
query = ("""select demo.PATID, ENCOUNTERID, RAW_PX, PX_DATE
            from [dbo].[PROCEDURES] as pro
         inner join [dbo].[vw_pc_dm_demo] as demo
         on pro.PATID = demo.PATID """)
df_procedures = pd.read_sql_query(query, engine)

In [14]:
query = ("""select demo.PATID, ENCOUNTERID, DX, ADMIT_DATE
            from [dbo].[DIAGNOSIS] as diag
         inner join [dbo].[vw_pc_dm_demo] as demo
         on diag.PATID = demo.PATID """)
df_diagnoses = pd.read_sql_query(query, engine)

## Define Clinical Profile Calculations and Writing Function

In [211]:
def calculateProfile(df_demographics, df_labs, df_diagnoses, df_meds, df_procedures,
                     cohort, medianEncounterYear=2019, sex='All', race='All', age_low='All', age_high=None):
    
    if (sex == 'All'):
        # grab whole dataframe
        df_sub_demographics = df_demographics
    else:
        # grab gender
        df_sub_demographics = df_demographics[df_demographics.SEX == sex]
        
    if (race != 'All'):
        # grab race
        df_sub_demographics = df_sub_demographics[df_sub_demographics.race_code == race]
        
    if (age_low != 'All'):
        # grab age
        dob_ub = medianEncounterYear - float(age_low)
        dob_lb = medianEncounterYear - float(age_high)
        df_sub_demographics = (df_sub_demographics[
            (pd.to_datetime(df_sub_demographics.BIRTH_DATE).dt.year >= dob_lb) & 
            (pd.to_datetime(df_sub_demographics.BIRTH_DATE).dt.year <= dob_ub)])

    # Initialize  profile
    clinicalProfile = ClinicalProfile()
    clinicalProfile.resourceType = 'ClinicalProfile'
    
    # Header info
    if (age_low != 'All'):
        clinicalProfile.identifier  = [identifier.Identifier({'value': 
                                                              'Group/'+cohort+'-'+sex+'-'+race+'-'+str(int(age_low))+'-'+str(int(age_high))})]
        clinicalProfile.cohort = fhirreference.FHIRReference({'reference': 
                                                      'Group/'+cohort+'-'+sex+'-'+race+'-'+str(int(age_low))+'-'+str(int(age_high))}) 
    else:
        clinicalProfile.identifier  = [identifier.Identifier({'value': 
                                                              'Group/'+cohort+'-'+sex+'-'+race+'-'+str(age_low)})]
        clinicalProfile.cohort = fhirreference.FHIRReference({'reference': 
                                                      'Group/'+cohort+'-'+sex+'-'+race+'-'+str(age_low)})
    clinicalProfile.status = 'draft'
    clinicalProfile.population = fhirreference.FHIRReference({'reference': 'Group/'+cohort})
     
    clinicalProfile.date = fhirdate.FHIRDate(str(datetime.now()).replace(' ', 'T'))
    clinicalProfile.reporter = fhirreference.FHIRReference({'reference': 'Organization/JHM',
                           'type': 'Organization',
                           'display': 'Johns Hopkins School of Medicine'})
    
    # Lab info

    df_labs_full = df_labs.merge(df_sub_demographics, on='PATID', how='right')

    # Calculate first
    labs_counts = df_labs_full.LAB_LOINC.value_counts()
    labs_codes = labs_counts.index
    df_labs_full['resultYear'] = pd.to_datetime(df_labs_full.RESULT_DATE).dt.year
    grouped_labs = df_labs_full.groupby(['LAB_LOINC', 'resultYear'])
    
    labs_frequencyPerYear = (df_labs_full.groupby(['LAB_LOINC','PATID','resultYear']).PATID.size()
                                    .groupby(['LAB_LOINC','resultYear']).aggregate(np.mean))
    labs_fractionOfSubjects = (np.divide(df_labs_full.groupby(['LAB_LOINC']).PATID.nunique(),
                                              df_labs_full.PATID.nunique()))
    labs_units = df_labs_full.groupby(['LAB_LOINC']).RAW_UNIT.unique()
    labs_names = df_labs_full.groupby(['LAB_LOINC']).SHORTNAME.unique()
    
    def percentile(n):
        def percentile_(x):
            return x.quantile(n*0.01)
        percentile_.__name__ = '%s' % n
        return percentile_
    
    labs_stats = (grouped_labs
               .RESULT_NUM.agg(['min','max', 'mean','median','std',
                                   percentile(10), percentile(20), percentile(30),
                                   percentile(40), percentile(50), percentile(60),
                                   percentile(70), percentile(80), percentile(90)]))
   
    def fracsAboveBelowNormal(x):
        try:
            aboveNorm = np.divide(np.sum(x.RESULT_NUM > x.range_high), x.RESULT_NUM.size)
            belowNorm = np.divide(np.sum(x.RESULT_NUM < x.range_low), x.RESULT_NUM.size)
            return pd.Series({'aboveNorm':aboveNorm, 'belowNorm':belowNorm})
        except:
            return pd.Series({'aboveNorm':np.nan, 'belowNorm':np.nan})
    
    df_labs_full['range_high'] = (pd.to_numeric(df_labs_full.range.dropna()
               .astype('str').str.split(',',expand=True)[1]).astype('float'))

    df_labs_full['range_low'] = (pd.to_numeric(df_labs_full.range.dropna()
               .astype('str').str.split(',',expand=True)[0]).astype('float'))
    
    labs_aboveBelowNorm = (grouped_labs.apply(fracsAboveBelowNormal))

    labs_correlatedLabsCoefficients = (df_labs_full.groupby(['LAB_LOINC','resultYear','PATID']).RESULT_NUM.mean())
    
    labs_abscorrelation = 0
   
    ## LABS TO MEDS
    # Need medication info to do correlation
    df_meds_full = df_meds.merge(df_sub_demographics, on='PATID', how='right')
    df_meds_full['startYear'] = pd.to_datetime(df_meds_full.RX_START_DATE).dt.year
    rxInfo = df_meds_full[['JH_INGREDIENT_RXNORM_CODE','PATID', 'startYear']]
    
    
    def patientsAboveBelowNormalMeds(x):
        # Get patients above and below normal
        patientsAboveNorm = x.PATID[x.RESULT_NUM > x.range_high].tolist()
        patientsBelowNorm = x.PATID[x.RESULT_NUM < x.range_low].tolist()
        
        # Get unique patient IDs for above & below normal
        patientsAboveBelowNorm = list(set(patientsAboveNorm + patientsBelowNorm))
        
        # Link to meds table
        abnormalPatientsMeds = rxInfo[rxInfo.PATID.isin(patientsAboveBelowNorm) &
                                     (rxInfo.startYear == pd.to_datetime(x.RESULT_DATE).dt.year.unique()[0])]
                        
        return pd.Series({'medsAboveBelowNorm': abnormalPatientsMeds.JH_INGREDIENT_RXNORM_CODE.value_counts().index,
                         'counts': abnormalPatientsMeds.JH_INGREDIENT_RXNORM_CODE.value_counts().values})
    
    # Need to grab the indices of those with abnormal lab, grab their medications, count and rank them 
    labs_correlatedMedsCoefficients = (grouped_labs.apply(patientsAboveBelowNormalMeds))
    
    # Currently a little hacky, but seems fast
    mytups = list()
    multiIndex = list()

    for lab in labs_correlatedMedsCoefficients.index:
        thisLabYear = labs_correlatedMedsCoefficients.loc[lab]
        thisLab = lab[0]
        thisYear = lab[1]
        totalCrossTab = np.sum(thisLabYear.counts)
        for medInd in range(len(labs_correlatedMedsCoefficients.loc[lab].medsAboveBelowNorm.values)):
            mytups.append((thisLabYear.medsAboveBelowNorm.values[medInd], thisLabYear.counts[medInd]/totalCrossTab))
            multiIndex.append((thisLab, thisYear))
        
    index = pd.MultiIndex.from_tuples(multiIndex)
    labs_correlatedMedsCoefficients = (pd.DataFrame.from_records(mytups, columns=['JH_INGREDIENT_RXNORM_CODE','Relative_Counts'],
                                                   index=index))
    
    ## LABS TO PROCEDURES
    df_procedures_full = (df_procedures.merge(df_sub_demographics, on='PATID', how='right'))
    df_procedures_full['encounterYear'] = pd.to_datetime(df_procedures_full.PX_DATE).dt.year
    procInfo = df_procedures_full[['RAW_PX','PATID', 'encounterYear']]
    
    def patientsAboveBelowNormalProcs(x):
        # Get patients above and below normal
        patientsAboveNorm = x.PATID[x.RESULT_NUM > x.range_high].tolist()
        patientsBelowNorm = x.PATID[x.RESULT_NUM < x.range_low].tolist()
        
        # Get unique patient IDs for above & below normal
        patientsAboveBelowNorm = list(set(patientsAboveNorm + patientsBelowNorm))
        
        # Link to procs table
        abnormalPatientsProcs = procInfo[procInfo.PATID.isin(patientsAboveBelowNorm) &
                                     (procInfo.encounterYear == pd.to_datetime(x.RESULT_DATE).dt.year.unique()[0])]
                        
        return pd.Series({'procsAboveBelowNorm': abnormalPatientsProcs.RAW_PX.value_counts().index,
                         'counts': abnormalPatientsProcs.RAW_PX.value_counts().values})
    
    # Need to grab the indices of those with abnormal lab, grab their medications, count and rank them 
    labs_correlatedProceduresCoefficients = (grouped_labs.apply(patientsAboveBelowNormalProcs))
    
    # Currently a little hacky, but seems fast
    mytups = list()
    multiIndex = list()

    for lab in labs_correlatedProceduresCoefficients.index:
        thisLabYear = labs_correlatedProceduresCoefficients.loc[lab]
        thisLab = lab[0]
        thisYear = lab[1]
        totalCrossTab = np.sum(thisLabYear.counts)
        for procInd in range(len(labs_correlatedProceduresCoefficients.loc[lab].procsAboveBelowNorm.values)):
            mytups.append((thisLabYear.procsAboveBelowNorm.values[procInd], thisLabYear.counts[procInd]/totalCrossTab))
            multiIndex.append((thisLab, thisYear))
            
    index = pd.MultiIndex.from_tuples(multiIndex)    
    labs_correlatedProceduresCoefficients = (pd.DataFrame.from_records(mytups, columns=['RAW_PX','Relative_Counts'],
                                                                      index=index))
    
    ## LABS TO DIAGNOSES
    df_diagnoses_full = (df_diagnoses.merge(df_sub_demographics, on='PATID', how='right'))
    df_diagnoses_full['admitYear'] = pd.to_datetime(df_diagnoses_full.ADMIT_DATE).dt.year
    diagInfo = df_diagnoses_full[['DX','PATID','admitYear']]
    
    def patientsAboveBelowNormalDiags(x):
        # Get patients above and below normal
        patientsAboveNorm = x.PATID[x.RESULT_NUM > x.range_high].tolist()
        patientsBelowNorm = x.PATID[x.RESULT_NUM < x.range_low].tolist()
        
        # Get unique patient IDs for above & below normal
        patientsAboveBelowNorm = list(set(patientsAboveNorm + patientsBelowNorm))
        
        # Link to procs table
        abnormalPatientsDiags = diagInfo[diagInfo.PATID.isin(patientsAboveBelowNorm) &
                                     (diagInfo.admitYear == pd.to_datetime(x.RESULT_DATE).dt.year.unique()[0])]
                        
        return pd.Series({'diagsAboveBelowNorm': abnormalPatientsDiags.DX.value_counts().index,
                         'counts': abnormalPatientsDiags.DX.value_counts().values})
    
    # Need to grab the indices of those with abnormal lab, grab their medications, count and rank them 
    labs_correlatedDiagnosisCoefficients = (grouped_labs.apply(patientsAboveBelowNormalDiags))
    
    # Currently a little hacky, but seems fast
    mytups = list()
    multiIndex = list()

    for lab in labs_correlatedDiagnosisCoefficients.index:
        thisLabYear = labs_correlatedDiagnosisCoefficients.loc[lab]
        thisLab = lab[0]
        thisYear = lab[1]
        totalCrossTab = np.sum(thisLabYear.counts)
        for diagInd in range(len(labs_correlatedDiagnosisCoefficients.loc[lab].diagsAboveBelowNorm.values)):
            mytups.append((thisLabYear.diagsAboveBelowNorm.values[diagInd], thisLabYear.counts[diagInd]/totalCrossTab))
            multiIndex.append((thisLab, thisYear))
    
    index = pd.MultiIndex.from_tuples(multiIndex)
    
    labs_correlatedDiagnosisCoefficients = (pd.DataFrame.from_records(mytups, columns=['DX','Relative_Counts'],
                                                                     index=index))
    """
    # Medication info
    
    meds_medication = df_meds_full.RXNorm.unique()

    uniqDropNA = lambda x: np.unique(x.dropna())

    #meds_dosageInfo = (df_meds_full.groupby('RXNorm')
    #              .agg({'Route':uniqDropNA, 'Dose':uniqDropNA,'Quantity':uniqDropNA}))

    df_meds_full['startYear'] = pd.to_datetime(df_meds_full.Start_date).dt.year

    meds_frequencyPerYear = (df_meds_full.groupby(['RXNorm','startYear','PatientID']).PatientID
                        .count().groupby(['RXNorm','startYear']).mean())

    meds_fractionOfSubjects = (np.divide(df_meds_full.groupby(['RXNorm']).PatientID.nunique(),
                                    df_meds_full.PatientID.nunique()))
    
    grouped_meds = df_meds_full.groupby(['RXNorm'])
    labInfo = df_labs_full[['Result_numeric','Loinc_Code','range_high','range_low','PatientID']]
    
    def medsPatientsAboveBelowNormal(x):
        
        patientsWithThisRX = list(set(x.PatientID.tolist()))
        
        # Get patients above and below normal
        #patientsAboveNorm = x.PatientID[x.Result_numeric > x.range_high].tolist()
        #patientsBelowNorm = x.PatientID[x.Result_numeric < x.range_low].tolist()
        #
        # Get unique patient IDs for above & below normal
        #patientsAboveBelowNorm = list(set(patientsAboveNorm + patientsBelowNorm))
        
        # Link to labs table
        abnormalPatientsLabs = labInfo[(labInfo.PatientID.isin(patientsWithThisRX)) & 
                                       ((labInfo.Result_numeric > labInfo.range_high) | (labInfo.Result_numeric < labInfo.range_low))]
                        
        return pd.Series({'labsAboveBelowNorm': abnormalPatientsLabs.Loinc_Code.value_counts().index,
                         'counts': abnormalPatientsLabs.Loinc_Code.value_counts().values})

    meds_correlatedLabsCoefficients = (grouped_meds.apply(medsPatientsAboveBelowNormal))
    
    # Currently a little hacky, but seems fast
    mytups = list()

    for med in meds_correlatedLabsCoefficients.index:
        thisMed = meds_correlatedLabsCoefficients.loc[med]
        totalCrossTab = np.sum(thisMed.counts)
        for labInd in range(len(meds_correlatedLabsCoefficients.loc[med].labsAboveBelowNorm.values)):
            mytups.append((med, thisMed.labsAboveBelowNorm.values[labInd], thisMed.counts[labInd]/totalCrossTab))
    
    meds_correlatedLabsCoefficients = (pd.DataFrame.from_records(mytups, columns=['RXNorm','Loinc_Code','Relative_Counts'])
                                                   .set_index('RXNorm'))
    
    #meds_correlatedLabsCoefficients
    """
    # Diagnosis info
    
    diagnoses_code = df_diagnoses_full.DX.unique()
    
    diagnoses_counts = df_diagnoses_full.DX.value_counts()

    diagnoses_frequencyPerYear = (df_diagnoses_full.groupby(['DX','admitYear','PATID']).PATID
                        .count().groupby(['DX','admitYear']).mean())

    diagnoses_fractionOfSubjects = (np.divide(df_diagnoses_full.groupby(['DX']).PATID.nunique(),
                                    df_diagnoses_full.PATID.nunique()))
    
    # Procedure info
   
    procedures_code = df_procedures_full.RAW_PX.unique()
    procedures_counts = df_procedures_full.RAW_PX.value_counts()
    
    procedures_frequencyPerYear = (df_procedures_full.groupby(['RAW_PX','encounterYear','PATID']).PATID.count()
                                            .groupby(['RAW_PX','encounterYear']).mean())

    procedures_fractionOfSubjects = (np.divide(df_procedures_full.groupby(['RAW_PX']).PATID.nunique(),
                                    df_procedures_full.PATID.nunique()))
                        
    
    # HPO info
    
    return (clinicalProfile, labs_counts, labs_frequencyPerYear, labs_fractionOfSubjects, labs_units, labs_names,
           labs_stats, labs_aboveBelowNorm, labs_correlatedLabsCoefficients, labs_abscorrelation,labs_correlatedMedsCoefficients,
    labs_correlatedProceduresCoefficients, labs_correlatedDiagnosisCoefficients,
            diagnoses_code, diagnoses_counts,
            diagnoses_frequencyPerYear, diagnoses_fractionOfSubjects, procedures_code, procedures_counts,
            procedures_frequencyPerYear, procedures_fractionOfSubjects)

In [16]:
def writeProfile(clinicalProfile, labs_counts, labs_frequencyPerYear, labs_fractionOfSubjects, labs_units, labs_names,
           labs_stats, labs_aboveBelowNorm, labs_correlatedLabsCoefficients, labs_abscorrelation,
                 labs_correlatedMedsCoefficients,labs_correlatedProceduresCoefficients, 
                 labs_correlatedDiagnosisCoefficients, diagnoses_code, diagnoses_counts,
            diagnoses_frequencyPerYear, diagnoses_fractionOfSubjects, procedures_code, procedures_counts,
            procedures_frequencyPerYear, procedures_fractionOfSubjects, cohort, sex='All', race='All', age_low='All', 
                 age_high=None):
    ## LABS
    labs = list()
    corrmat = (pd.DataFrame(labs_correlatedLabsCoefficients).unstack(level=[0,1]).corr()
                        .droplevel(level=0).droplevel(level=0,axis=1))
    lab_names = pd.DataFrame({'lab_name':labs_names}).reset_index()
    lab_counts = pd.DataFrame({'lab_counts':labs_counts}).reset_index().rename({'index':'LAB_LOINC'},axis=1)
    lab_info = lab_names.merge(lab_counts, how='inner', on='LAB_LOINC').set_index('LAB_LOINC')

    for thisLab in lab_info.index:
        
        # Check if STDEV is NaN and skip that lab if so
        if np.isnan(float(labs_stats.loc[thisLab]['std'].median())):
            continue
        
        # Build the profile
        thisCPLab = ClinicalProfileLab()
        try:
            thisCPLab.code = [codeableconcept.CodeableConcept(dict(coding=[dict(system='https://loinc.org', 
                                                                                code=thisLab)],
                                                                  text=lab_info.loc[thisLab]['lab_name'][0]))]
            thisCPLab.count = int(lab_info.loc[thisLab]['lab_counts'])
            thisCPLab.frequencyPerYear = round(float(labs_frequencyPerYear.loc[thisLab].mean()),3)
            thisCPLab.fractionOfSubjects = round(float(labs_fractionOfSubjects.loc[thisLab].mean()),3)
            thisCPLab.scalarDistribution = ClinicalProfileLabScalarDistribution()
            thisCPLab.scalarDistribution.units = quantity.Quantity(dict(unit=str(labs_units.loc[thisLab][0])))
            thisCPLab.scalarDistribution.min = round(float(labs_stats.loc[thisLab]['min'].min()),3)
            thisCPLab.scalarDistribution.max = round(float(labs_stats.loc[thisLab]['max'].max()),3)
            thisCPLab.scalarDistribution.mean = round(float(labs_stats.loc[thisLab]['mean'].mean()),3)
            thisCPLab.scalarDistribution.median = round(float(labs_stats.loc[thisLab]['median'].median()),3)
            thisCPLab.scalarDistribution.stdDev = round(float(labs_stats.loc[thisLab]['std'].median()),3)
            deciles = list()
            for dec in labs_stats.columns[5:]:
                deciles.append(ClinicalProfileLabScalarDistributionDecile(
                                                                    dict(nth=int(dec), 
                                                                        value=round(labs_stats.loc[thisLab][dec].mean(),3))))
            thisCPLab.scalarDistribution.decile = deciles

            thisCPLab.scalarDistribution.fractionAboveNormal = round(float(labs_aboveBelowNorm.loc[thisLab].aboveNorm.mean()),3)
            thisCPLab.scalarDistribution.fractionBelowNormal = round(float(labs_aboveBelowNorm.loc[thisLab].belowNorm.mean()),3)

            yearly_vals = dict()
            for year in corrmat.loc[thisLab].index:
                crosstab = corrmat.loc[(thisLab, year)]
                yearly_vals[year] = crosstab[crosstab.index.get_level_values(level=1) == year].droplevel(level=1)

            top10corrs = pd.DataFrame(yearly_vals).apply(np.mean, axis=1).drop(thisLab).nlargest(10).round(3)

            entries = list()
            for code, corr in top10corrs.iteritems():
                otherLoinc = [(dict(coding=[dict(system='https://loinc.org', code=code)],
                                                                  text=lab_info.loc[code]['lab_name'][0]))]
                entries.append(dict(labcode=otherLoinc, coefficient=corr))

            try:
                thisCPLab.scalarDistribution.correlatedLabs = ClinicalProfileLabScalarDistributionCorrelatedLabs(
                                                                dict(topn=10, 
                                                                     entry=entries))
            except:
                print('No correlated Labs for Lab ', thisLab)

            try:
                top10corrs = (pd.DataFrame(labs_correlatedMedsCoefficients.loc[thisLab].groupby(['RXNorm'])
                                                                                    .Relative_Counts.mean())
                                                                                    .Relative_Counts.nlargest(10).round(3))
                entries = list()
                for code, corr in top10corrs.iteritems():
                    otherRX = [(dict(coding=[dict(system='https://www.nlm.nih.gov/research/umls/rxnorm/', code=code)]))]
                    entries.append(dict(medicationCodeableConcept=otherRX, coefficient=corr))

                thisCPLab.scalarDistribution.correlatedMedications = ClinicalProfileLabScalarDistributionCorrelatedMedications(
                                                                        dict(topn=10, 
                                                                          entry=entries))
            except:
                print('No correlated Meds for Lab ', thisLab)

            try:
                top10corrs = (pd.DataFrame(labs_correlatedDiagnosisCoefficients.loc[thisLab].groupby(['DX'])
                                                                                    .Relative_Counts.mean())
                                                                                    .Relative_Counts.nlargest(10).round(3))
                entries = list()
                for code, corr in top10corrs.iteritems():
                    otherDX = [(dict(coding=[dict(system='https://www.icd10data.com/', code=code)]))]
                    entries.append(dict(code=otherDX, coefficient=corr))

                thisCPLab.scalarDistribution.correlatedDiagnoses = ClinicalProfileLabScalarDistributionCorrelatedDiagnoses(
                                                                        dict(topn=10, 
                                                                          entry=entries))
            except:
                print('No correlated Diagnoses for Lab ', thisLab)

            try:      
                top10corrs = (pd.DataFrame(labs_correlatedProceduresCoefficients.loc[thisLab].groupby(['RAW_PX'])
                                                                                    .Relative_Counts.mean())
                                                                                    .Relative_Counts.nlargest(10).round(3))
                entries = list()
                for code, corr in top10corrs.iteritems():
                    otherProc = [(dict(coding=[dict(system='https://www.ama-assn.org/practice-management/cpt', code=code)]))]
                    entries.append(dict(code=otherProc, coefficient=corr))

                thisCPLab.scalarDistribution.correlatedProcedures = ClinicalProfileLabScalarDistributionCorrelatedProcedures(
                                                                        dict(topn=10, 
                                                                          entry=entries))
            except:
                print('No correlated Procedures for Lab ', thisLab)
            
            labs.append(thisCPLab)
        
        except:
            print('This lab did not work ', thisLab)
        

    ## MEDICATIONS
    """meds = list()
    for thisMed in meds_medication[~np.isnan(meds_medication)]:
        thisCPMed = ClinicalProfileMedication()
        thisCPMed.medication = [codeableconcept.CodeableConcept(dict(
                                                    coding=[dict(
                                                        system='https://www.nlm.nih.gov/research/umls/rxnorm/', 
                                                        code=str(thisMed))]))]
        try:
            thisCPMed.frequencyPerYear = round(float(meds_frequencyPerYear.loc[thisMed].mean()),3)
            thisCPMed.fractionOfSubjects = round(float(meds_fractionOfSubjects.loc[thisMed].mean()),3)
        except:
            print('No frequency per year for med ', thisMed)

        try:
            top10corrs = (pd.DataFrame(meds_correlatedLabsCoefficients.loc[thisMed].groupby(['Loinc_Code'])
                                                                                .Relative_Counts.mean())
                                                                                .Relative_Counts.nlargest(10).round(3))
            entries = list()
            for code, corr in top10corrs.iteritems():
                otherLab = [(dict(coding=[dict(system='https://loinc.org', code=code)]))]
                entries.append(dict(labcode=otherLab, coefficient=corr))


            thisCPMed.correlatedLabs = ClinicalProfileLabScalarDistributionCorrelatedLabs(dict(topn=10, entry=entries))
        except:
            print('No correlated Labs for Med ', thisMed)

        try:
            top10corrs = (pd.DataFrame(meds_correlatedDxCoefficients.loc[thisMed].groupby(['icd_10'])
                                                                                .Relative_Counts.mean())
                                                                                .Relative_Counts.nlargest(10).round(3))
            entries = list()
            for code, corr in top10corrs.iteritems():
                otherDX = [(dict(coding=[dict(system='https://www.icd10data.com/', code=code)]))]
                entries.append(dict(code=otherDX, coefficient=corr))

            thisCPMed.correlatedDiagnoses = ClinicalProfileLabScalarDistributionCorrelatedDiagnoses(dict
                                                                                                    (topn=10, 
                                                                                                     entry=entries))
        except:
            print('No correlated DX for Med ', thisMed)

        meds.append(thisCPMed)"""

    ## DIAGNOSES 
    dxs = list()
    for thisDX in diagnoses_code:
        thisCPdx = ClinicalProfileDiagnosis()
        try:
            thisCPdx.code = [codeableconcept.CodeableConcept(dict(coding=[dict(
                                                            system='https://www.icd10data.com/', 
                                                            code=str(thisDX))]))]
            thisCPdx.count = int(diagnoses_counts.loc[thisDX])

            thisCPdx.frequencyPerYear = round(float(diagnoses_frequencyPerYear.loc[thisDX].mean()),3)
            thisCPdx.fractionOfSubjects = round(float(diagnoses_fractionOfSubjects.loc[thisDX].mean()),3)
            dxs.append(thisCPdx)
        except:
            print('This DX did not work ', thisDX)


    ## PROCEDURES
    procs = list()
    for thisProc in procedures_code:
        thisCPProc = ClinicalProfileProcedure()
        try:
            thisCPProc.code = [codeableconcept.CodeableConcept(dict(coding=[dict(
                                                            system='https://www.ama-assn.org/practice-management/cpt', 
                                                            code=str(thisProc))]))]

            thisCPProc.frequencyPerYear = round(float(procedures_frequencyPerYear.loc[thisProc].mean()),3)
            thisCPProc.fractionOfSubjects = round(float(procedures_fractionOfSubjects.loc[thisProc].mean()),3)
            procs.append(thisCPProc)
        except:
            print('This procedure did not work ', thisProc)


    clinicalProfile.lab = labs
    clinicalProfile.diagnosis = dxs
    clinicalProfile.procedure = procs

    if age_high != 'nan':
        filename = cohort+'_resources/'+cohort+'-'+sex+'-'+race+'-'+age_low+'-'+age_high+'.json'
    else:
        filename = cohort+'_resources/'+cohort+'-'+sex+'-'+race+'-'+age_low+'.json'
        
    with open(filename, 'w') as outfile:
        json.dump(clinicalProfile.as_json(), outfile, indent=4)
    
    del(clinicalProfile)
    return print('Write to '+ filename + ' successful')

In [17]:
cohortDefinitions

,gender,race,age_low,age_high
0,All,All,All,NaN
1,All,All,18,44.0
2,All,All,45,64.0
3,All,All,65,74.0
4,All,All,75,200.0
5,All,White or Caucasian,All,NaN
6,All,White or Caucasian,18,44.0
7,All,White or Caucasian,45,64.0
8,All,White or Caucasian,65,74.0
9,All,White or Caucasian,75,200.0


## Investigation of Females vs. Males of "Other" Race and Ages 45-64

### 1\. Calculate Profiles for each and Use DataFrames to Compare Two Sub-Cohorts 

In [18]:
(clinicalProfile_female, labs_counts_female, labs_frequencyPerYear_female, labs_fractionOfSubjects_female, labs_units_female,
 labs_names_female, labs_stats_female, labs_aboveBelowNorm_female, labs_correlatedLabsCoefficients_female, 
 labs_abscorrelation_female,labs_correlatedMedsCoefficients_female, labs_correlatedProceduresCoefficients_female, 
 labs_correlatedDiagnosisCoefficients_female, diagnoses_code_female, diagnoses_counts_female,
 diagnoses_frequencyPerYear_female, diagnoses_fractionOfSubjects_female, procedures_code_female, procedures_counts_female,
            procedures_frequencyPerYear_female, procedures_fractionOfSubjects_female) = calculateProfile(
                        df_demographics, df_labs, df_diagnoses, df_meds, df_procedures, 
                            'diabetes', sex='F', race='Other', age_low=45, age_high=64)

In [19]:
(clinicalProfile_male, labs_counts_male, labs_frequencyPerYear_male, labs_fractionOfSubjects_male, labs_units_male,
 labs_names_male, labs_stats_male, labs_aboveBelowNorm_male, labs_correlatedLabsCoefficients_male, 
 labs_abscorrelation_male,labs_correlatedMedsCoefficients_male, labs_correlatedProceduresCoefficients_male, 
 labs_correlatedDiagnosisCoefficients_male, diagnoses_code_male, diagnoses_counts_male,
 diagnoses_frequencyPerYear_male, diagnoses_fractionOfSubjects_male, procedures_code_male, procedures_counts_male,
            procedures_frequencyPerYear_male, procedures_fractionOfSubjects_male) = calculateProfile(
                        df_demographics, df_labs, df_diagnoses, df_meds, df_procedures, 
                            'diabetes', sex='M', race='Other', age_low=45, age_high=64)

### 2\. Compare Male and Female Lab Counts, Diagnosis Counts, Procedure Counts

In [56]:
import plotly.plotly as py
import plotly.graph_objs as go

In [207]:
layout = go.Layout(barmode='group', xaxis={'type':'category', 'title':'Loinc Code'},
                  yaxis={'title':'Count'}, title='Lab Counts For Males and Females in This Cohort')
fig = go.FigureWidget(layout=layout)
fig.add_bar(x=labs_counts_female[:20].index, y=labs_counts_female[:20], name='Female', marker={'color':'#d8b365'})
fig.add_bar(x=labs_counts_male[:20].index, y=labs_counts_male[:20], name='Male', marker={'color':'#5ab4ac'})
fig

FigureWidget({
    'data': [{'marker': {'color': '#d8b365'},
              'name': 'Female',
              'ty…

In [208]:
layout = go.Layout(barmode='group', xaxis={'type':'category', 'title':'Diagnosis Code'},
                  yaxis={'title':'Count'}, title='Diagnosis Counts For Males and Females in This Cohort')
fig = go.FigureWidget(layout=layout)
fig.add_bar(x=diagnoses_counts_female[:20].index, y=diagnoses_counts_female[:20], name='Female', 
            marker={'color':'#d8b365'})
fig.add_bar(x=diagnoses_counts_male[:20].index, y=diagnoses_counts_male[:20], name='Male', 
            marker={'color':'#5ab4ac'})
fig

FigureWidget({
    'data': [{'marker': {'color': '#d8b365'},
              'name': 'Female',
              'ty…

In [209]:
layout = go.Layout(barmode='group', xaxis={'type':'category', 'title':'Procedure Code'},
                  yaxis={'title':'Count'}, title='Procedure Counts For Males and Females in This Cohort')
fig = go.FigureWidget(layout=layout)
fig.add_bar(x=procedures_counts_female[:20].index, y=procedures_counts_female[:20], name='Female', 
            marker={'color':'#d8b365'})
fig.add_bar(x=procedures_counts_male[:20].index, y=procedures_counts_male[:20], name='Male', marker={'color':'#5ab4ac'})
fig

FigureWidget({
    'data': [{'marker': {'color': '#d8b365'},
              'name': 'Female',
              'ty…

### 3\. Choose Some Lab and Compare Disitributions Across Years For Males and Females

In [187]:
trial_male = labs_stats_male.loc['10501-5'][['min','max','10','20','30','40','50','60','70','80','90']].reset_index().transpose()

trial_male.columns = trial_male.loc['resultYear']

trial_male.drop('resultYear', inplace=True)

trial_male = trial_male[2016].append(trial_male[2017]).append(trial_male[2018]).append(trial_male[2019])

trial_male = pd.DataFrame({'values': trial_male, 'year': [2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,
                                       2017,2017,2017,2017,2017,2017,2017,2017,2017,2017, 2017,
                                       2018,2018,2018,2018,2018,2018,2018,2018,2018,2018, 2018,
                                       2019,2019,2019,2019,2019,2019,2019,2019,2019,2019, 2019]})

In [188]:
trial_female = labs_stats_female.loc['10501-5'][['min','max','10','20','30','40','50','60','70','80','90']].reset_index().transpose()

trial_female.columns = trial_female.loc['resultYear']

trial_female.drop('resultYear', inplace=True)

trial_female = trial_female[2016].append(trial_female[2017]).append(trial_female[2018]).append(trial_female[2019])

trial_female = pd.DataFrame({'values': trial_female, 'year': [2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,
                                       2017,2017,2017,2017,2017,2017,2017,2017,2017,2017, 2017,
                                       2018,2018,2018,2018,2018,2018,2018,2018,2018,2018, 2018,
                                       2019,2019,2019,2019,2019,2019,2019,2019,2019,2019, 2019]})

In [194]:
layout = go.Layout(xaxis={'type':'category', 'title':'Year'}, boxmode='group', title='Lutropin [Units/volume] in Serum or Plasma',
                  yaxis={'title':'Distribution [Units/volume]'})
fig = go.FigureWidget(layout=layout)

fig.add_box(x= trial_male.year, y=trial_male['values'], name='Male', marker={'color':'#5ab4ac'})
fig.add_box(x= trial_female.year, y=trial_female['values'], name='Female', marker={'color':'#d8b365'})

fig

FigureWidget({
    'data': [{'marker': {'color': '#5ab4ac'},
              'name': 'Male',
              'type…

### 4\. For A Lab and Year, Compare the Procedure Counts for Those Patients

In [202]:
labprocs_male = labs_correlatedProceduresCoefficients_male.loc[('5902-2',2016.0)]
labprocs_female = labs_correlatedProceduresCoefficients_female.loc[('5902-2',2016.0)]

In [206]:
layout = go.Layout(barmode='group', xaxis={'type':'category', 'title':'Procedure Codes'},
                  yaxis={'title':'Relative Frequency'},title='Relative Frequency Counts of Prothrombin Time with Procedures in 2016')
fig = go.FigureWidget(layout=layout)
fig.add_bar(x=labs_correlatedProceduresCoefficients_female.loc[('5902-2',2016.0)][:20].RAW_PX, 
            y=labs_correlatedProceduresCoefficients_female.loc[('5902-2',2016.0)][:20].Relative_Counts, name='Female', 
            marker={'color':'#d8b365'})
fig.add_bar(x=labs_correlatedProceduresCoefficients_male.loc[('5902-2',2016.0)][:20].RAW_PX, 
            y=labs_correlatedProceduresCoefficients_male.loc[('5902-2',2016.0)][:20].Relative_Counts, name='Male', 
            marker={'color':'#5ab4ac'})
fig

FigureWidget({
    'data': [{'marker': {'color': '#d8b365'},
              'name': 'Female',
              'ty…

In [7]:
query="""select * from [dbo].[jh_hpo_loinc]"""
df_hpo = pd.read_sql_query(query, engine)

In [9]:
medianEncounterYear=2019
df_sub_demographics = df_demographics[df_demographics.SEX == 'F']
df_sub_demographics = df_sub_demographics[df_sub_demographics.race_code == 'Other']
dob_ub = medianEncounterYear - float(45)
dob_lb = medianEncounterYear - float(64)
df_sub_demographics = (df_sub_demographics[
    (pd.to_datetime(df_sub_demographics.BIRTH_DATE).dt.year >= dob_lb) & 
    (pd.to_datetime(df_sub_demographics.BIRTH_DATE).dt.year <= dob_ub)])

In [ ]:
for ind, cohort in cohortDefinitions[cohortDefinitions.gender == 'M'].iterrows():
    (clinicalProfile, labs_counts, labs_frequencyPerYear, labs_fractionOfSubjects, labs_units, labs_names,
           labs_stats, labs_aboveBelowNorm, labs_correlatedLabsCoefficients, labs_abscorrelation,labs_correlatedMedsCoefficients,
    labs_correlatedProceduresCoefficients, labs_correlatedDiagnosisCoefficients,
            diagnoses_code, diagnoses_counts,
            diagnoses_frequencyPerYear, diagnoses_fractionOfSubjects, procedures_code, procedures_counts,
            procedures_frequencyPerYear, procedures_fractionOfSubjects) = calculateProfile(df_demographics, 
                     df_labs, df_diagnoses, df_meds, df_procedures, 'diabetes', sex=cohort.gender, race=cohort.race, 
                    age_low=cohort.age_low, age_high=cohort.age_high)
    

    writeProfile(clinicalProfile, labs_counts, labs_frequencyPerYear, labs_fractionOfSubjects, labs_units, labs_names,
           labs_stats, labs_aboveBelowNorm, labs_correlatedLabsCoefficients, labs_abscorrelation,
                 labs_correlatedMedsCoefficients,labs_correlatedProceduresCoefficients, 
                 labs_correlatedDiagnosisCoefficients, diagnoses_code, diagnoses_counts,
            diagnoses_frequencyPerYear, diagnoses_fractionOfSubjects, procedures_code, procedures_counts,
            procedures_frequencyPerYear, procedures_fractionOfSubjects, 'diabetes', sex=cohort.gender, race=cohort.race, 
                 age_low=str(cohort.age_low),age_high=str(cohort.age_high))